In [1]:
#pip install C:\Users\Aman\Downloads\gmpy2-2.0.8-cp37-cp37m-win_amd64.whl
import gmpy2 as mp
import numpy as np
import math

vigKeyLen = 0
vigKey = ""
pKey_A = ""
sKey_A = ""
pKey_B = ""
sKey_B = ""
charsToNum = {}
numToChars = {}
charSpace = 29

In [158]:
def readFile(fname):
    '''Read File from the directory'''
    try:
        text = ""
        with open(fname,'r') as openFile:
            t = openFile.read().replace('\n','')
        openFile.close()
        return t.lower()
    except:
        print("No such file found in directory. Please check")
        return

def writeFile(fname, text):
    '''write to File from the directory'''
    try:
        with open(fname,'w') as openFile:
            openFile.write(text)
        openFile.close()
        return
    except:
        print("No such file found in directory. Please check")
        return

def charsDictGen():  
    '''#form dictionary of chars a-z and map them to 0-25 numbers'''
    charsDict = {"a":None}
    init = 'a'
    for i in range(0,26):
        charsDict[init] = i
        init = chr(ord(init) + 1)
    charsDict[' '] = 26
    charsDict[','] = 27
    charsDict['.'] = 28
    return charsDict

def numbersDictGen():  
    '''#form dictionary of 0-25 numbers and map them to chars a-z '''
    numsDict = {0:None}
    init = 'a'
    for i in range(0,26):
        numsDict[i] = init
        init = chr(ord(init) + 1)
    numsDict[26] = ' '
    numsDict[27] = ','
    numsDict[28] = '.'
    return numsDict

def getMsgBlocks(text, l):
    '''Returns list of message blocks of length l'''
    msgBlocks = []
    tempBlock = ""
    
    for char in text:
        tempBlock += char
        if(len(tempBlock) == l):
            msgBlocks.append(tempBlock)
            tempBlock = ""
    msgBlocks.append(tempBlock) 
    return msgBlocks

def enVigenere(text, key):
    '''Encrypt the given message from vigenere cipher.'''
    
    msgBlocks = getMsgBlocks(text, len(key))
    enMsg = ""
    for block in msgBlocks:
        for i in range(len(block)):
            enMsg += numToChars[(charsToNum[block[i]] + charsToNum[key[i]])%charSpace]
    
    return enMsg

def deVigenere(text, key):
    
    msgBlocks = getMsgBlocks(text, len(key))
    deMsg = ""
    for block in msgBlocks:
        for i in range(len(block)):
            deMsg += numToChars[(charsToNum[block[i]] - charsToNum[key[i]])%charSpace]
    
    return deMsg

def genStrongPrime(bits):
    '''Generates strong prime based on BPSW Strong probable prime test'''
    notStrongPrime = True
    prp = mp.next_prime(2**bits)
    count = 0
    while notStrongPrime:
        if mp.is_strong_bpsw_prp(prp):
            notStrongPrime = False
        elif not mp.is_strong_bpsw_prp(prp):
            prp = mp.next_prime(prp)
    return prp

def getPQ(b):
    '''returns p and q for rsa for generating n = p*q of min length "b" bits'''
    b = round(b/2) + int(np.random.choice([0], 1)[0])
    p = genStrongPrime(b)
    
    b += int(np.random.choice([1], 1)[0])
    q = genStrongPrime(b)
    return p,q

def rsaKeyGen(bits):
    '''RSA Private and Public Key Generator'''
    p, q = getPQ(bits)
    #print("P,Q: ",p, q)
    n = mp.mul(p,q)
    #print("N: ",n)
    #print("\nbit length of n", mp.bit_length(n))
    phi = mp.mul(p-1, q-1)
    #print("Phi: ",phi)
    e = 2 + 1
    notCoprime = True
    while notCoprime:
        if mp.gcd(e, phi) == 1:
            notCoprime = False
        else:
            e += 2
    
    d = mp.invert(e, phi)
    
    #print(mp.t_mod(mp.mul(e,d), phi) == 1)
    return e, n, d

    
def enRSA(text, sendTo):    
    receiverPubKey = readFile("public_" + sendTo + ".txt").split()
    e = int(receiverPubKey[0])
    n = int(receiverPubKey[1])
    
    msgBlocks = getMsgBlocks(text, e)
    if(len(msgBlocks[-1]) != e):
        for i in range(e - len(msgBlocks[-1])):
            msgBlocks[-1] += numToChars[np.random.randint(0, 27)]
        print(msgBlocks[-1])
    cipherMsgBlocks = []
    for block in msgBlocks:
        m = 0
        for i in range(len(block)):
            m += mp.mul(charsToNum[block[i]],(mp.mpz(charSpace)**(e-i)))
        
        c = mp.powmod(m, e, n)
        print("c: ", c)
        cipherMsgBlocks.append(c)
    cipherMsg = ""
    for msg in cipherMsgBlocks:
        c = ""
        rem = msg
        for i in range(e):
            div = mp.mpz(charSpace)**(e-i)
            quot, rem = mp.t_divmod(rem, div)
            char = mp.t_mod(quot, charSpace)
                        
            c += numToChars[char]
            rem = div + rem
            
        cipherMsg += c
        print("cMsg:", c)
    return cipherMsg

def deRSA(text, receiver):
    receiverSecKey = readFile("secret_" + receiver + ".txt").split()
    e = int(receiverSecKey[0])
    d = mp.mpz(receiverSecKey[1])
    n = mp.mpz(receiverSecKey[2])
    print("Len to Decrypt", len(text), n)
    msgBlocks = getMsgBlocks(text, e)
    
    deMsgBlocks = []
    for block in msgBlocks:        
        if len(block) != 0:
            c = mp.mpz(0)
            for i in range(e):
                mult = mp.mpz(charSpace)**(e-i)
                c += mp.mul(charsToNum[block[i]], mult)
            
            tempMsg = mp.powmod(c, d, n)
            deMsgBlocks.append(tempMsg)
            print("m: ", tempMsg)
            
    decryptedMsg = "" 
    for msg in deMsgBlocks:
        m = ""
        rem = msg
        for j in range(e):
            div = mp.mpz(charSpace)**(e-j)
            quot, rem = mp.t_divmod(mp.mpz(rem), mp.mpz(div))
            char = mp.t_mod(quot, charSpace)

            m += numToChars[char]
            rem = div + rem
        decryptedMsg += m
        print("dMsg: ", m)
    
    return decryptedMsg


    
def enRSA_N(text, sendTo):    
    receiverPubKey = readFile("public_" + sendTo + ".txt").split()
    e = int(receiverPubKey[0])
    n = int(receiverPubKey[1])
    print("ENCRYPTION e,n",e,n )
    msgBlocks = getMsgBlocks(text, e)
    if(len(msgBlocks[-1]) != e):
        for i in range(e - len(msgBlocks[-1])):
            msgBlocks[-1] += numToChars[np.random.randint(0, 27)]
        print("MSG BLOCKS:",msgBlocks)
    cipherMsgBlocks = []
    for block in msgBlocks:
        m = 0
        for i in range(len(block)):
            m += mp.mul(charsToNum[block[i]],(mp.mpz(charSpace)**(e-i)))
            print("m", m)
        
        c = mp.powmod(m, e, n)
        print("c: ", c)
        cipherMsgBlocks.append(c)
    cipherMsg = cipherMsgBlocks
    '''for msg in cipherMsgBlocks:
        c = ""
        rem = msg
        for i in range(e):
            div = mp.mpz(charSpace)**(e-i)
            quot, rem = mp.t_divmod(rem, div)
            char = mp.t_mod(quot, charSpace)
                        
            c += numToChars[char]
            
        cipherMsg += c
        print("cMsg:", c)'''
    return cipherMsg

def deRSA_N(text, receiver):
    receiverSecKey = readFile("secret_" + receiver + ".txt").split()
    e = int(receiverSecKey[0])
    d = mp.mpz(receiverSecKey[1])
    n = mp.mpz(receiverSecKey[2])
    print("DECRYPTION e,d,n",e,d,n )
    
    #msgBlocks = getMsgBlocks(text, e)
    
    deMsgBlocks = []
    
    decryptedMsg = "" 
    for msg in text:
        m = ""
        rem = mp.powmod(msg, d, n)
        for j in range(e):
            div = mp.mpz(charSpace)**(e-j)
            quot, rem = mp.t_divmod(mp.mpz(rem), mp.mpz(div))
            char = mp.t_mod(quot, charSpace)

            m += numToChars[char]
            rem = div + rem
        decryptedMsg += m
        print("dMsg: ", m)
    
    return decryptedMsg
   

In [157]:
charsToNum = charsDictGen()
numToChars = numbersDictGen()
vigKey = readFile("vigenere_key.txt")
vigKeyLen = len(vigKey)


e, n, d = rsaKeyGen(2)

writeFile("public_a.txt", str(e) + " " + str(n))
writeFile("secret_a.txt", str(e) + " " + str(d) + " " + str(n))
e, n, d = rsaKeyGen(3)

writeFile("public_b.txt", str(e) + " " + str(n))
writeFile("secret_b.txt", str(e) + " " + str(d) + " " + str(n))

enMsg_RSA = enRSA_N("abcd", "b")
print("enMsg_RSA\n Len of msg = ", len(enMsg_RSA), "\n", enMsg_RSA)

deMsg_RSA = deRSA_N(enMsg_RSA, 'b')
print("MESSAGE - RSA DECRYPTED\n", deMsg_RSA)

ENCRYPTION e,n 3 55
MSG BLOCKS: ['abc', 'drj']
m 0
m 841
m 899
c:  39
m 73167
m 87464
m 87725
c:  0
cMsg: aab
cMsg: aaa
enMsg_RSA
 Len of msg =  6 
 aabaaa
DECRYPTION e,d,n 3 27 55
m:  39
m:  0
dMsg:  aab
dMsg:  aaa
MESSAGE - RSA DECRYPTED
 aabaaa


In [130]:
charsToNum = charsDictGen()
numToChars = numbersDictGen()
vigKey = readFile("vigenere_key.txt")
vigKeyLen = len(vigKey)

msg = readFile("message.txt")
enMsg_V = enVigenere(msg, vigKey)
#print("MESSAGE - VIGENERE ENCRYPTED\n", enMsg_V, len(enMsg_V))

deMsg_V = deVigenere(enMsg_V, vigKey)
#print("MESSAGE - VIGENERE DECRYPTED\n", deMsg_V, len(deMsg_V))

e, n, d = rsaKeyGen(512)

#writeFile("public_a.txt", str(e) + " " + str(n))
#writeFile("secret_a.txt", str(e) + " " + str(d) + " " + str(n))
e, n, d = rsaKeyGen(512)

#writeFile("public_b.txt", str(e) + " " + str(n))
#writeFile("secret_b.txt", str(e) + " " + str(d) + " " + str(n))

enMsg_RSA = enRSA("enMsg_V", "b")
print("enMsg_RSA\n Len of msg = ", len(enMsg_RSA), "\n", enMsg_RSA)

deMsg_RSA = deRSA_NEW(enMsg_RSA, 'b')
print("MESSAGE - RSA DECRYPTED\n", deMsg_RSA)
print("MESSAGE - VIGENERE ENCRYPTED\n", enMsg_V, len(enMsg_V))

P,Q:  115792089237316195423570985008687907853269984665640564039457584007913129640233 231584178474632390847141970017375815706539969331281128078915168015826259280027
N:  26815615859885194199148049996411692254958731641184786755447122887443528060233822228442498426706061523151570959355071320222072548089446870314794232112526291
Phi:  26815615859885194199148049996411692254958731641184786755447122887443528060233474852174786478119790810196544895631511510268075626397328497562770492723606032
True
P,Q:  115792089237316195423570985008687907853269984665640564039457584007913129640233 231584178474632390847141970017375815706539969331281128078915168015826259280027
N:  26815615859885194199148049996411692254958731641184786755447122887443528060233822228442498426706061523151570959355071320222072548089446870314794232112526291
Phi:  26815615859885194199148049996411692254958731641184786755447122887443528060233474852174786478119790810196544895631511510268075626397328497562770492723606032
True
c:  39
c:  36
cMsg

TypeError: powmod() argument types not supported

In [98]:
msg = 'abc'
print(msg)
enMsg_RSA = enRSA(msg, "b")
print("enMsg_RSA\n Len of msg = ", len(enMsg_RSA), "\n", enMsg_RSA)

deMsg_RSA = deRSA_NEW(enMsg_RSA, 'b')
print("MESSAGE - RSA DECRYPTED\n", len(deMsg_RSA), "\n", deMsg_RSA)

abc
c:  74914146013694001972484998634780049314243735604366323866690283818857721147393354187783143740337838062435650709920088186966034096265157546565405450913911064390523
enMsg_RSA
 Len of msg =  1 
 [mpz(74914146013694001972484998634780049314243735604366323866690283818857721147393354187783143740337838062435650709920088186966034096265157546565405450913911064390523)]
Len to Decrypt 1 1098367625620897554397104127853022914763109648022928865503114153469686909344141984649737728811046790187270601678718101898270989824013402700910536966945909560313
dMsg:  xcibmijjwfm.htpx inaqslmvqdpgpyjqnrswlkzyjszhnbpmsvvgrkmltvcptqwy
MESSAGE - RSA DECRYPTED
 65 
 xcibmijjwfm.htpx inaqslmvqdpgpyjqnrswlkzyjszhnbpmsvvgrkmltvcptqwy


In [60]:
c = mp.mpz(1807387693030948128258639516319666249939379272549655780222084927263898875814482695261068212348045690)
d = mp.mpz(18852578648716898321741339507925020178769792466065196945202705619255820085757643599763097258595464196201843653853520367038886735134913812319323674016772638947)
n = mp.mpz(5491838128104487771985520639265114573815548240114644327515570767348434546720710895902238237511275508932627081372016476959226140500804944948258351305018523720701)

In [91]:
m = mp.mpz(50992409245153330347817232148178742068576577889381756340804721165523046691354075902944238497844136062926760927340480378253669284392226071645316595663979127672)
c = mp.mpz(86286039801835347119868576825611309868432362412008843862226684437308596336907262701408373110492335638484004573410777561377605673439392711269342982189029311780544)
n = mp.mpz(1098367625620897554397104127853022914763109648022928865503114153469686909344141984649737728811046790187270601678718101898270989824013402700910536966945909560313)

print(mp.t_mod(mp.add(m, c), n))

664357412650491207242272085223701258978386444109774109324585187838540454755541974631774501728530139939824403398106093690722136450739526669966415362912345203802


3 5
True
5 11
True
c:  490050
c:  537139
enMsg_RSA
 Len of msg =  2 
 [mpz(490050), mpz(537139)]
Len to Decrypt 2 5557314633
3 27 5557314633
dMsg:  ltb
dMsg:  gzd
MESSAGE - RSA DECRYPTED
 ltbgzd
MESSAGE - VIGENERE ENCRYPTED
 wicw 4
